In [ ]:
from math import sqrt

from bs4 import BeautifulSoup

from pyspark import SparkConf, SparkContext
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF
from pyspark.mllib.recommendation import *
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
import pyspark.sql.functions as func


In [ ]:
# 將 1.txt~100.txt 讀出並接成一個大 list

test = []
x = 1
while x < 11 :

    try:        
        f = open('/home/jupyter/notebooks/test/' + str(x) + '.txt', 'r')
        f = f.read()
        test.append(f)
        x = x + 1
    except IOError:
        print("No such file or directory: {filename}".format(filename=str(x)))
        x = x + 1
        continue

In [ ]:
sc = SparkContext()

In [ ]:
# Spark 的進入點
test1= sc.parallelize(test)

In [ ]:
# 將 list 中的每個值(文本)依據空格將個詞分隔成值在接成一個大 list
test2 = test1.flatMap(lambda x: x.split(' '))
test2 = test2.distinct()

In [ ]:
# 將 list 中的每個值(文本)依據空格將個詞分隔成值再依據文本分割成不同的 list
test1 = test1.map(lambda x: x.split(' '))
test1_collect = test1.collect()

In [ ]:
# 宣告Hash表的大小
hashingTF = HashingTF()

In [ ]:
# 計算TF值
tf = hashingTF.transform(test1)

In [ ]:
# 找出每個詞的 hash index 值
tf1 = test2.map(lambda x: [x,hashingTF.indexOf(x)])
c = tf1.collect()

In [ ]:
tf.cache()

In [ ]:
# 計算tfidf值
idf = IDF(minDocFreq=5).fit(tf)
tfidf = idf.transform(tf)
d = tfidf.collect()
print(d)

In [ ]:
# 取出 n 篇文章 (n 個 list) 的 index 表
tfidf_index = tfidf.take(10)

In [ ]:
# 用全部詞的大 list 放到第一張 index 表 (tfidf_index[0]) 中找出值
tmp1 = c
y = 0
while y < len(c) :
    try:        
        tmp1[y].append(tfidf_index[7][tmp1[y][1]])
        y = y + 1
    except IndexError:
        print("list index out of range: {index}".format(index=str(y))) 
        y = y + 1
        continue

In [ ]:
# 去掉 0 的值就得出第一篇文章有哪些詞
word_1 = sc.parallelize(tmp1)
bb  = word_1.filter(lambda x: x[2] != 0.0 )
bbb = bb.filter(lambda x : x[2] > 2)
bbbb = bbb.collect()
print(bbbb)
for i in bbbb :
    x = i[2]
    fid = open('/home/jupyter/notebooks/test/0804.txt', 'a') 
    fid.write(str(x)+' '+ str(i[0])+'\n')
    fid.close()

In [ ]:
cat /home/jupyter/notebooks/test/0804.txt

In [ ]:
rm /home/jupyter/notebooks/test/0804.txt